In [1]:
%load_ext autoreload
%autoreload complete

In [2]:
import torch
import importlib as imp
import layers
import tensorflow as tf

2023-09-10 14:41:37.766745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
"""parallelization example!"""
def div(x : torch.Tensor):
    return x + 1
a = torch.Tensor([1])
b = torch.Tensor([2])
c = torch.Tensor([3])
d = torch.Tensor([4])

f = lambda x : div(x)
e, f, g, h = torch.vmap(func=div)(torch.stack([a, b, c, d]))
print(e)
print(f)
print(g)
print(h)

tensor([2.])
tensor([3.])
tensor([4.])
tensor([5.])


In [26]:
def div_tuple(x : torch.Tensor):
    a, b, c, d = x
    return (a + 1, b + 1, c + 1, d + 1)
torch.vmap(func=div_tuple)((a, b, c, d))

(tensor([2.]), tensor([3.]), tensor([4.]), tensor([5.]))

In [4]:
number_of_images = 7
batch_of_images_list = []
for ct in range(number_of_images):
    image = torch.randint(0, 255, (32, 32, 3))
    batch_of_images_list.append(image)
batch_of_images = torch.stack(batch_of_images_list)
print(f"batch of images: {batch_of_images.size()}")
batch_of_labels = torch.stack([torch.randint(0, 24, (1, 1)) for i in range(number_of_images)])
print(f"batch of labels: {batch_of_labels.size()}")

batch of images: torch.Size([7, 32, 32, 3])
batch of labels: torch.Size([7, 1, 1])


In [13]:
kernel_size_0 = 5
kernel_size_1 = 5
out_channels = 3
weight = torch.zeros(size=[out_channels, 3, kernel_size_0, kernel_size_1])
bias = torch.zeros(size=[out_channels])
l = layers.CONV(batch_of_images, out_channels, weight, bias)
l.bias.data

tensor([0., 0., 0.])

In [22]:
a = torch.Tensor([
    [1, 2, 3],
    [4, 5, 6]
])
var, mean = torch.var_mean(a, dim=1)
print(a.size())
print(var.size())
print(mean.size())

torch.Size([2, 3])
torch.Size([2])
torch.Size([2])


In [7]:
import tensorflow as tf
a_t = tf.constant([
    [1, 2, 3],
    [4, 5, 6]
])
mean_t, var_t = tf.nn.moments(x=a_t, axes=0)
print(a_t)
print(var_t)
print(mean_t)

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
tf.Tensor([2 2 2], shape=(3,), dtype=int32)
tf.Tensor([2 3 4], shape=(3,), dtype=int32)


In [24]:
import encoder
imp.reload(encoder)
ways = 3
shots = 5
model = encoder.InferenceNetwork(ways=ways, shots=shots, data_name='cifar',
                         need_g=True, need_o=True, need_s=True, need_z=True)
omega, gamma, zeta, KL = model(batch_of_images.float(), batch_of_labels.float(), True)
print(f"omega: {omega.size()}, corresponding to ways: {ways}")
print(f"gamma: ")
for key in gamma.keys():
    print(f"\t{key}: {gamma[key].size()}")
print(f"zeta: ")
for key in zeta.keys():
    print(f"\t{key}: {zeta[key].size()}")
print(f"KL: {KL}")

omega: torch.Size([3]), corresponding to ways: 3
gamma: 
	convolution1_weight: torch.Size([1])
	convolution1_bias: torch.Size([1])
	convolution2_weight: torch.Size([1])
	convolution2_bias: torch.Size([1])
	convolution3_weight: torch.Size([1])
	convolution3_bias: torch.Size([1])
	convolution4_weight: torch.Size([1])
	convolution4_bias: torch.Size([1])
	dense_weight: torch.Size([1])
	dense_bias: torch.Size([1])
zeta: 
	convolution1_weight: torch.Size([32])
	convolution1_bias: torch.Size([32])
	convolution2_weight: torch.Size([32])
	convolution2_bias: torch.Size([32])
	convolution3_weight: torch.Size([32])
	convolution3_bias: torch.Size([32])
	convolution4_weight: torch.Size([32])
	convolution4_bias: torch.Size([32])
KL: 3075.778564453125
